In [1]:
import pandas as pd
import cv2

In [2]:
df = pd.read_csv("../data/mirror-data.csv")
df["Action"] = df["Action"].str.rstrip()
df = df[df.Action != "Talking&Yawning"]
df["label"] = df.Action.astype('category').cat.codes

In [3]:
i = 0
dfTrain = pd.DataFrame()
dfTest = pd.DataFrame()

while i<len(df):
    if i%5==0:
        dfTest = pd.concat([dfTest, df.iloc[[i]]])
    else :
        dfTrain = pd.concat([dfTrain, df.iloc[[i]]])
    i+=1

In [4]:
dfTest.head()

,Action,video-name,label
0,Normal,1-FemaleNoGlasses-Normal.avi,0
5,Yawning,2-FemaleNoGlasses-Yawning.avi,2
10,Talking,4-FemaleGlasses-Talking.avi,1
15,Normal,6-FemaleNoGlasses-Normal.avi,0
20,Yawning,7-FemaleGlasses-Yawning.avi,2


In [5]:
root = "../data/YawDD/YawDD dataset/Mirror/all/"

In [38]:
test_labeled_video_path = list(zip(root+dfTest["video-name"], dfTest["Action"]))
train_labeled_video_path = list(root+dfTrain["video-name"])

In [39]:
train_labeled_video_path

['../data/YawDD/YawDD dataset/Mirror/all/1-FemaleNoGlasses-Talking.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/1-FemaleNoGlasses-Yawning.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/2-FemaleNoGlasses-Normal.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/2-FemaleNoGlasses-Talking.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/3-FemaleGlasses-Normal.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/3-FemaleGlasses-Talking.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/3-FemaleGlasses-Yawning.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/4-FemaleGlasses-Normal.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/4-FemaleGlasses-Yawning.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/5-FemaleGlasses-Normal.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/5-FemaleGlasses-Talking.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/5-FemaleGlasses-Yawning.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/6-FemaleNoGlasses-Talking.avi',
 '../data/YawDD/YawDD dataset/Mirror/all/6-FemaleNoGlasses-Yawning.avi'

In [24]:
MAX_SEQ_LENGTH = 1448
fps = 30
clip_duration = MAX_SEQ_LENGTH / fps

In [8]:
from pytorchvideo.transforms import ApplyTransformToKey
from torchvision.transforms import Compose, Lambda, Resize

c:\Users\xuton\Documents\Work\Year 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [9]:
IMG_SIZE = 128
resize_to = (IMG_SIZE, IMG_SIZE)

In [10]:
train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    Lambda(lambda x: x / 255.0),
                    Resize(resize_to),
                ]
            ),
        ),
    ]
)

In [11]:
import pytorchvideo.data

In [40]:
train_dataset = pytorchvideo.data.LabeledVideoDataset(
    labeled_video_paths=train_labeled_video_path,
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", 10),
    decode_audio=False,
    transform=train_transform
)

In [27]:
test_dataset = pytorchvideo.data.LabeledVideoDataset(
    labeled_video_paths=test_labeled_video_path,
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", 10),
    decode_audio=False,
    transform=train_transform
)

In [41]:
next(iter(train_dataset))

RuntimeError: Failed to load video after 10 retries.

In [30]:
import imageio
import numpy as np
from IPython.display import Image

def unnormalize_img(img):
    """Un-normalizes the image pixels."""
    img = (img * 255).astype("uint8")
    return img.clip(0, 255)

def create_gif(video_tensor, filename="sample.gif"):
    """Prepares a GIF from a video tensor.
    
    The video tensor is expected to have the following shape:
    (num_frames, num_channels, height, width).
    """
    frames = []
    for video_frame in video_tensor:
        frame_unnormalized = unnormalize_img(video_frame.permute(1, 2, 0).numpy())
        frames.append(frame_unnormalized)
    kargs = {"duration": 0.25}
    imageio.mimsave(filename, frames, "GIF", **kargs)
    return filename

def display_gif(video_tensor, gif_name="sample.gif"):
    """Prepares and displays a GIF from a video tensor."""
    video_tensor = video_tensor.permute(1, 0, 2, 3)
    gif_filename = create_gif(video_tensor, gif_name)
    return Image(filename=gif_filename)

sample_video = next(iter(train_dataset))
video_tensor = sample_video["video"]
display_gif(video_tensor)

RuntimeError: Failed to load video after 10 retries.

In [1]:
label2id = {"normal": 0, "talking": 1, "yawning": 2}
id2label = {i: label for label, i in label2id.items()}

In [1]:
from transformers import TimesformerForVideoClassification

In [ ]:
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")